## 手寫數字辨識

### 手寫數字辨識程式碼
```
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential()
model.add(Dense(20, input_dim=784, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='mse', optimizer=SGD(learning_rate=0.05), metrics=['accuracy'])

model.fit(x_train, y_train, validation_split=0.2, batch_size=100, epochs=20)

predict = np.argmax(model.predict(x_test), axis=-1)

score = model.evaluate(x_test, y_test)
print('正確率', score[1])

model.save('number.h5')
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist

In [ ]:
n = 1000
plt.imshow(x_test[n].reshape(28,28), cmap='binary')
print('預測為:{}，正確為：{}'.format(predict[n], y_test[n]))

## gradio：網頁互動界面

### gradio程式碼
**手寫數字**
```
!pip install gradio

import gradio as gr
import tensorflow

model = tensorflow.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/number.h5")

def mnist(image):
    image = image.reshape(1, 784)
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

grobj = gr.Interface(fn=mnist, inputs="sketchpad", outputs=gr.outputs.Label(num_top_classes=3, label='預測結果'), title="手寫數字")
grobj.launch()

from urllib.request import urlretrieve
urlretrieve("https://gr-models.s3-us-west-2.amazonaws.com/mnist-model.h5", "mnist-model.h5")
model = load_model("mnist-model.h5")
```
**Inception辨識物體**
```
import gradio as gr
import tensorflow
import numpy as np
import requests

model = tensorflow.keras.applications.InceptionV3()

response = requests.get('https://git.io/JJkYN')

labels = response.text.split('\n')

def classify(img):
    img = np.expand_dims(img, 0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    prediction = model.predict(img)[0]
    return {labels[i]: float(prediction[i]) for i in range(len(prediction))

imputs = gr.inputs.Image(shape=(299, 299))

outputs = gr.outputs.Label(num_top_classes=3, label='預測結果')

grobj = gr.Interface(fn=classify, inputs=imputs, outputs=outputs, title='Inception物件偵測')
grobj.launch()
```
**自動歌詞產生器(繁體)** 
```
import gradio as gr
grobj = gr.Interface.load("huggingface/uer/gpt2-chinese-lyric", inputs="text", outputs="text")
grobj.launch()

!pip install opencc
!pip install transformers

import gradio as gr
from opencc import OpenCC
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline

tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-lyric")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-lyric")

cc = OpenCC('s2twp')

def lyric(text):
    text_generator = TextGenerationPipeline(model, tokenizer)
    ret = text_generator(text, max_length=100, do_sample=True)
    return cc.convert(ret[0]['generated_text'])

grobj = gr.Interface(lyric,inputs="textbox", outputs=gr.outputs.Textbox(), title="自動產生歌詞")
grobj.launch()
```

### 手寫數字

In [ ]:
import gradio as gr
import tensorflow

In [ ]:
model = tensorflow.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/number.h5")

In [ ]:
from urllib.request import urlretrieve

In [ ]:
urlretrieve("https://gr-models.s3-us-west-2.amazonaws.com/mnist-model.h5", "mnist-model.h5")
model = load_model("mnist-model.h5")

### Inception辨識物體

In [ ]:
import gradio as gr
import tensorflow
import numpy as np
import requests

In [ ]:
grobj = gr.Interface(fn=classify, inputs=imputs, outputs=outputs, title='Inception物件偵測')

In [ ]:
grobj.launch()

### 自動歌詞產生器(繁體)

In [ ]:
import gradio as gr
grobj = gr.Interface.load("huggingface/uer/gpt2-chinese-lyric", inputs="text", outputs="text")
grobj.launch()

In [ ]:
!pip install opencc
!pip install transformers

In [ ]:
import gradio as gr
from opencc import OpenCC
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline

In [ ]:
grobj = gr.Interface(lyric,inputs="textbox", outputs=gr.outputs.Textbox(), title="自動產生歌詞")
grobj.launch()

## Teachable Machine

```
import numpy as np
import cv2
from time import sleep
import tensorflow.keras

labels = ['cloth','fist','nothing']
current_x = 300
move = 10
model = tensorflow.keras.models.load_model('keras_model.h5')

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
while True:
    success, image = cap.read()
    if success == True:
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        image = cv2.flip(image,1) #左右反轉
        img = cv2.resize(image,(224,224))
        img = np.array(img,dtype=np.float32)
        img = np.expand_dims(img,axis=0)
        img = (img/127.0) - 1 #正規化
        data[0] = img

        prediction = model.predict(data) #預測
        predicted_class = np.argmax(prediction[0], axis=-1)
        predicted_class_name = labels[predicted_class]
        print(current_x, predicted_class_name)

        if predicted_class_name == 'cloth':
            current_x -= move
            if current_x < 0:
                current_x = 0
        elif predicted_class_name == 'fist':
            current_x += move
            if current_x > width:
                current_x = width
        cv2.putText(image, 'O', (current_x,100), cv2.FONT_HERSHEY_PLAIN, 0.4, (255, 0, 0), cv2.LINE_AA)
        cv2.imshow("Frame",image)
        sleep(0.2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()	
```

## Bonus

In [ ]:
!pip install git+https://github.com/Zeecka/pytube@fix_1060#egg=pytube
!pip install google_trans_new

In [ ]:
!sed -i "s/response = (decoded_line + ']')/response = decoded_line/g" /usr/local/lib/python3.7/dist-packages/google_trans_new/google_trans_new.py

In [ ]:
from pytube import YouTube
from google_trans_new import google_translator
import re

translator = google_translator()
yt = YouTube('https://www.youtube.com/watch?v=3hRxS_As6-s')
fname = yt.title
fname = re.sub('[<>|?"*:/]', '', fname)  #去除檔名中特殊字元
fname = '/content/drive/MyDrive/Colab Notebooks/package/' + fname
#print(yt.captions)
if 'a.en' in yt.captions.keys():
  yt.streams.filter(subtype='mp4', res='720p', progressive=True).first().download('/content/drive/MyDrive/Colab Notebooks/package/')
  srt = yt.captions['a.en'].generate_srt_captions()
  srtlist = srt.split('\n')
  for i in range(2, len(srtlist), 4):
      srtlist[i] = translator.translate(srtlist[i], lang_src='en', lang_tgt='zh-TW')
  srt_zh = '\n'.join(srtlist)
  file = open(fname + '.srt', 'w', encoding='Big5')
  file.write(srt_zh)
  file.close()
  print('下載完成！')
else:
  print('沒有指定字幕！')